# Day 3: Gear Ratios

## Part 1

> <p>You and the Elf eventually reach a <a href="https://en.wikipedia.org/wiki/Gondola_lift" target="_blank">gondola lift</a> station; he says the gondola lift will take you up to the <em>water source</em>, but this is as far as he can bring you. You go inside.</p>
> <p>It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.</p>
> <p>"Aaah!"</p>
> <p>You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.</p>
> <p>The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can <em>add up all the part numbers</em> in the engine schematic, it should be easy to work out which part is missing.</p>
> <p>The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently <em>any number adjacent to a symbol</em>, even diagonally, is a "part number" and should be included in your sum. (Periods (<code>.</code>) do not count as a symbol.)</p>
> <p>Here is an example engine schematic:</p>
> <pre><code>467..114..
> ...*......
> ..35..633.
> ......#...
> 617*......
> .....+.58.
> ..592.....
> ......755.
> ...$.*....
> .664.598..
> </code></pre>


In [ ]:
from operator import mul
from re import Match, finditer

from advent23 import Checker, disp_name, get_chk, get_inp

chk = get_chk()
inp = get_inp(3)

SYMBOLS = "#$%&*+-/=@"
"""Possible symbols in the input.

Found by taking the intersection of symbols in `string.punctuation` and
`input/blake.txt`, excluding `.`. If your input includes other symbols, you
might need to include those, too. I doubt our inputs will carry different symbols,
though.
"""

ESCAPED_SYMBOLS = r"#\$%&\*\+\-\/=@"
"""Same as `SYMBOLS`, but with special regex characters escaped.

If you pursue a regex solution, forgetting to escape these characters will introduce
subtle bugs.
"""

ESCAPED_ASTERISK = r"\*"
"""Escaped asterisk for part two, if you pursue a regex solution."""

check = Checker(chk)
checks = {}

#### Part 1

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..



#### Part 2

<same as part 1>


In [ ]:
WIDTH = inp["a"].find("\n")
"""Width of the field for coordinate calculations."""

INP_NO_LN = inp["a"].replace("\n", "")
"""Input with newlines removed.

Avoids accumulating off-by-one errors when finding coordinates.
"""


def get_pos(sym: Match[str]) -> tuple[int, int, int]:
    """Get the position of a match."""
    start, after_end = sym.span()
    end = after_end - 1
    row, left = divmod(start, WIDTH)
    right = left + end - start
    return row, left, right


def get_pad(sym: Match[str]) -> tuple[int, int, int, int]:
    """Get the bounds of a one-element pad around a match."""
    row, left, right = get_pos(sym)
    return row - 1, row + 1, left - 1, right + 1

In [ ]:
numbers = list(finditer(r"\d+", INP_NO_LN))
symbol_positions = [
    get_pos(sym)[:2] for sym in finditer(rf"[{ESCAPED_SYMBOLS}]", INP_NO_LN)
]
chk["symbol_positions"] = [list(pos) for pos in symbol_positions]

#### Symbol positions

[[1, 3], [3, 6], [4, 3], [5, 5], [8, 3], [8, 5]]

In [ ]:
chk["not_parts"] = [114, 58]
inp["a"] = inp["a"]

#### Not parts

[114, 58]

#### Part 1

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..



> <p>In this schematic, two numbers are <em>not</em> part numbers because they are not adjacent to a symbol: <code>114</code> (top right) and <code>58</code> (middle right).


In [ ]:
parts: list[int] = []
for num in numbers:
    (top, bottom, left, right) = get_pad(num)
    for row, col in symbol_positions:
        if top <= row <= bottom and left <= col <= right:
            parts.append(int(num[0]))
            break

chk["parts"] = parts
if wrong_parts := [part for part in parts if part in chk["not_parts"]]:
    disp_name("Wrong parts", [part for part in parts if part in chk["not_parts"]])
chk["a"] = sum(parts)

#### Parts

[467, 35, 633, 617, 592, 755, 664, 598]

#### Part 1

4361

> Every other number is adjacent to a symbol and so <em>is</em> a part number; their sum is <code><em>4361</em></code>.</p>
>
> <p>Of course, the actual engine schematic is much larger. <em>What is the sum of all of the part numbers in the engine schematic?</em></p>
> </article>

# Part 2

> <p>The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.</p>
> <p>You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.</p>
> <p>Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.</p>
> <p>The missing part wasn't the only issue - one of the gears in the engine is wrong. A <em>gear</em> is any <code>*</code> symbol that is adjacent to <em>exactly two part numbers</em>. Its <em>gear ratio</em> is the result of <span title="They're magic gears.">multiplying</span> those two numbers together.</p>
> <p>This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.</p>
> <p>Consider the same engine schematic again:</p>
> <pre><code>467..114..
> ...*......
> ..35..633.
> ......#...
> 617*......
> .....+.58.
> ..592.....
> ......755.
> ...$.*....
> .664.598..
> </code></pre>


In [ ]:
gears: list[list[int]] = []
for row, col in [get_pos(sym)[:2] for sym in finditer(ESCAPED_ASTERISK, INP_NO_LN)]:
    neighbors: list[int] = []
    for num in numbers:
        (top, bottom, left, right) = get_pad(num)
        if top <= row <= bottom and left <= col <= right:
            neighbors.append(int(num.group()))
            if len(neighbors) > 1:
                gears.append(neighbors)
                break

In [ ]:
chk["first_gear"] = gears[0]
chk["first_gear_ratio"] = mul(*gears[0])
chk["second_gear"] = gears[1]
chk["second_gear_ratio"] = mul(*gears[1])
chk["sum_of_all_gear_ratios"] = sum(mul(*gear) for gear in gears)

#### First gear

[467, 35]

#### First gear ratio

16345

#### Second gear

[755, 598]

#### Second gear ratio

451490

#### Sum of all gear ratios

467835

> <p>In this schematic, there are <em>two</em> gears. The first is in the top left; it has part numbers <code>467</code> and <code>35</code>, so its gear ratio is <code>16345</code>. The second gear is in the lower right; its gear ratio is <code>451490</code>.


> (The <code>\*</code> adjacent to <code>617</code> is <em>not</em> a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces <code><em>467835</em></code>.</p>
>
> <p><em>What is the sum of all of the gear ratios in your engine schematic?</em></p>
> </article>
